In [ ]:
!pip install qiskit
!pip install pyscf
!pip install matplotlib

In [ ]:
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.chemistry.algorithms.ground_state_solvers.minimum_eigensolver_factories import VQEUCCSDFactory
from qiskit.chemistry.drivers import PySCFDriver, UnitsType, Molecule
from qiskit.chemistry.transformations import FermionicTransformation, FermionicQubitMappingType
from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.chemistry.algorithms.ground_state_solvers import GroundStateEigensolver

In [ ]:
# Set up the molcule, here we can feed data from OpenChemistry

molecule = Molecule(geometry=[['Li', [0., 0., 0.]],
                              ['Li', [0., 0., 2.67298]]],
                     charge=0, multiplicity=1)

In [ ]:
# Define the classical quantum chemistry driver to get the molecular integrals
# Also gives us the Hartree-Fock energy and orbital energies if desired

driver = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM, basis='sto3g')
print('Hartree-Fock energy = ', driver.run().hf_energy)

In [ ]:
# Generating a small quantum calculation freezing core and throwing away virtuals, 
# only keeping 2 electrons in 2 orbitals

transformation = FermionicTransformation(qubit_mapping=FermionicQubitMappingType.JORDAN_WIGNER, 
                                         freeze_core=True, orbital_reduction=list(range(4, 10)))

In [ ]:
# Quantum solver and wave function structure generator (UCCSD)

vqe_solver = VQEUCCSDFactory(QuantumInstance(BasicAer.get_backend('qasm_simulator')), 
                             include_custom=True, method_doubles='succ_full')

In [ ]:
# Run the smallest quantum simulation

calc = GroundStateEigensolver(transformation, vqe_solver)
res = calc.solve(driver)

print(res)

In [ ]:
# Generating a bit larger quantum calculation freezing core and throwing away some virtuals, 
# only keeping 2 electrons in 5 orbitals

transformation = FermionicTransformation(qubit_mapping=FermionicQubitMappingType.JORDAN_WIGNER, 
                                         freeze_core=True, orbital_reduction=list(range(7, 10)))

In [ ]:
# Run the bit larger quantum simulation

calc = GroundStateEigensolver(transformation, vqe_solver)
res = calc.solve(driver)

print(res)

In [ ]:
print(res.total_energies[0])

In [ ]:
res.__dict__